#### Peer-graded Assignment - Week 3 - All 3 questions are answered in this notebook.

In [ ]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests
!pip install folium

#  1. Create a dataframe from Wikipedia

### Get a local copy of the Wikipedia article

In [29]:
import requests
wiki_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

from bs4 import BeautifulSoup
soup = BeautifulSoup(wiki_url,"lxml")
#print(soup.prettify())

tables = soup.find("table",{"class":"wikitable sortable"})

### Span the rows and columns of wiki table and convert to a list

In [30]:
    # preinit list of lists
    rows = tables.findAll("tr")
    row_lengths = [len(r.findAll(['th', 'td'])) for r in rows]
    ncols = max(row_lengths)
    nrows = len(rows)
    data = []
    for i in range(nrows):
        rowD = []
        for j in range(ncols):
            rowD.append('')
        data.append(rowD)

    # process html
    for i in range(len(rows)):
        row = rows[i]
        rowD = []
        cells = row.findAll(["td", "th"])
        for j in range(len(cells)):
            cell = cells[j]

            #lots of cells span cols and rows so lets deal with that
            cspan = int(cell.get('colspan', 1))
            rspan = int(cell.get('rowspan', 1))
            l = 0
            for k in range(rspan):
                # Shifts to the first empty cell of this row
                while data[i + k][j + l]:
                    l += 1
                for m in range(cspan):
                    cell_n = j + l + m
                    row_n = i + k
                    # in some cases the colspan can overflow the table, in those cases just get the last item
                    cell_n = min(cell_n, len(data[row_n])-1)
                    data[row_n][cell_n] += cell.text
                    #print(cell.text)

        data.append(rowD)

### Convert the list to a dataframe - Table is generated after this code is run

In [31]:
import pandas as pd
import numpy as np
df = pd.DataFrame(data,columns=data[0])

df1 = df.drop([0])

df2 = df1[:287]

df3 = df2.rename(columns = {"Postcode": "PostalCode","Neighbourhood\n": "Neighbourhood"})
df3["Neighbourhood"] = df3["Neighbourhood"].str.replace(r'\n$', '')

df4 = df3[df3.Borough != "Not assigned"]
#df4[df4.PostalCode == "M6A"]
#df4[df4.Neighbourhood == "Not assigned"]
df4["Neighbourhood"] = np.where(df4["Neighbourhood"] == "Not assigned", df4["Borough"], df4["Neighbourhood"])

df5 = df4.groupby(["PostalCode","Borough"])["Neighbourhood"].apply(', '.join).reset_index()

df5

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [32]:
df5.shape

(103, 3)

# 2. Get the latitude and the longitude

In [33]:
LatLong = pd.read_csv("Geospatial_Coordinates.csv")

#join df5 table with LatLong to get Latitude and Longitude fields
df6 = df5.set_index("PostalCode").join(LatLong.set_index("Postal Code"))
df6

,Borough,Neighbourhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...
M9N,York,Weston,43.706876,-79.518188
M9P,Etobicoke,Westmount,43.696319,-79.532242
M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724


# 3. Explore and cluster the neighborhoods in Toronto

In [42]:
#filter for Borough that contain the word Toronto
df7 = df6[df6["Borough"].str.contains("Toronto")]
df7.reset_index(inplace=True)
df7

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [49]:
import folium
latitude = df7["Latitude"]
longitude = df7["Longitude"]

#address = 'Toronto'

#geolocator = Nominatim(user_agent="ny_explorer")
#location = geolocator.geocode(address)
#latitude = location.latitude
#longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 0     43.676357
1     43.679557
2     43.668999
3     43.659526
4     43.728020
5     43.712751
6     43.715383
7     43.704324
8     43.689574
9     43.686412
10    43.679563
11    43.667967
12    43.665860
13    43.654260
14    43.657162
15    43.651494
16    43.644771
17    43.657952
18    43.650571
19    43.640816
20    43.647177
21    43.648198
22    43.711695
23    43.696948
24    43.672710
25    43.662696
26    43.653206
27    43.628947
28    43.646435
29    43.648429
30    43.669542
31    43.669005
32    43.647927
33    43.636847
34    43.661608
35    43.648960
36    43.651571
37    43.662744
38    43.667856
Name: Latitude, dtype: float64, 0    -79.293031
1    -79.352188
2    -79.315572
3    -79.340923
4    -79.388790
5    -79.390197
6    -79.405678
7    -79.388790
8    -79.383160
9    -79.400049
10   -79.377529
11   -79.367675
12   -79.383160
13   -79.360636
14   -79.378937
15   -79.375418
16   -79.373306
17   -79.387383
18   -79.38456

In [50]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

ValueError: Location should consist of two numerical values, but 0     43.676357
1     43.679557
2     43.668999
3     43.659526
4     43.728020
5     43.712751
6     43.715383
7     43.704324
8     43.689574
9     43.686412
10    43.679563
11    43.667967
12    43.665860
13    43.654260
14    43.657162
15    43.651494
16    43.644771
17    43.657952
18    43.650571
19    43.640816
20    43.647177
21    43.648198
22    43.711695
23    43.696948
24    43.672710
25    43.662696
26    43.653206
27    43.628947
28    43.646435
29    43.648429
30    43.669542
31    43.669005
32    43.647927
33    43.636847
34    43.661608
35    43.648960
36    43.651571
37    43.662744
38    43.667856
Name: Latitude, dtype: float64 of type <class 'pandas.core.series.Series'> is not convertible to float.